# Final Statement

In [1]:
import pandas as pd
import numpy as np

In [2]:
features = pd.read_csv('features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


Для обучения фича start_time, а также шесть последних фич не нужны -- время начала матча ничего не значит, а результаты игры на момент пятой минуты фактически неизвестны

In [3]:
X = features.iloc[:,:-6]
X = X.drop(['start_time'], axis=1)
X.head(3)

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,0,35,103,-84,221,3,4,2,2,-52
1,0,42,4,1188,1033,9,0,1,12,49,...,0,-20,149,-84,195,5,4,3,1,-5
2,7,33,4,1319,1270,22,0,0,12,98,...,1,-39,45,-77,221,3,4,3,1,13


Найдем колонки, в которых имеются пропущенные данные (сделаем это не с помощью метода _count_):

In [4]:
_features_nan = features.dropna(axis=1)

In [5]:
for i in features.drop(_features_nan, axis=1).columns:
    print(i)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Исходя из специфики геймплея Dota 2, данные события не считаются чрезвычайными, т.к. вполне часто в матчах начального уровня игроки до конца игры не покупают, например, варды или ботл. Или же _first blood_ может случится позднее пятой минуты, это вполне нормальное дело

Перейдем к стандартному именованию переменных, где Х - фичи, а у - __целевая переменная__ (*radiant_win*)
По заданию требуется заменить эти данные на нулевые значения,

In [6]:
X = X.fillna(0)
y = features.radiant_win

## Подход 1. Градиентный бустинг "в лоб"

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import time
import datetime


import math
import time
import datetime

from sklearn.cross_validation import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline

Для удобства поделим выборку на части с помощью *train_test_split* и используем поиск по сетке параметров (GridSearchCV):

*Подробнее по ссылке: http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html *

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.5, 
                                                    random_state = 1)

In [9]:
cv = KFold(n=len(X_train),n_folds=5,random_state=1,shuffle=True)
clf = GradientBoostingClassifier()

params = {'n_estimators':[10, 20, 30, 40]}
_grid = GridSearchCV(clf, 
                        params,
                        cv = cv,
                        scoring = 'roc_auc'
                       )
_grid.fit(X_train, y_train)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=48615, n_folds=5, shuffle=True, random_state=1),
       error_score='raise',
       estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [10]:
for params, mean_score, scores in _grid.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()

0.663 (+/-0.017) for {'n_estimators': 10}
0.680 (+/-0.015) for {'n_estimators': 20}
0.688 (+/-0.015) for {'n_estimators': 30}
0.693 (+/-0.015) for {'n_estimators': 40}



Ради эксперимента затестили на 40 деревьях что получится, замечаем, что качество дальше растет, что можно продолжать проверять, однако, это достаточно затратно со в плане мощностей и времени работы

Узнаем как долго проводилась кросс-валидация на 30 элементах:

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [12]:
grid_30 = GradientBoostingClassifier(n_estimators=30)
start_time = datetime.datetime.now()
scr = cross_val_score(estimator = grid_30,
                     X = X_train,
                     y = y_train,
                      cv=cv,
                      scoring = 'roc_auc')

print('Time elapsed:', datetime.datetime.now() - start_time)
print(scr.mean())

Time elapsed: 0:00:29.686200
0.688041165747


Проверим как же будет идти подсчет на всей выборке

In [13]:
scr2 = cross_val_score(estimator = grid_30,
                     X = X,
                     y = y,
                      cv=cv,
                      scoring = 'roc_auc')

print('Time elapsed:', datetime.datetime.now() - start_time)
print(scr2.mean())

Time elapsed: 0:00:59.206232
0.699296198064


Результаты гораздо круче (т.к. данных для обучения в два раза больше), таким образом, чтобы менять точность достаточно либо меня количество данных, либо меня количество *n_estimators*, что также повлияет на результат

## Подход 2. Логистическая регрессия

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

Проведем нормализацию признаков, из-за чувствительности логистической регрессии к масштабу

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kf = KFold(n=len(X_scaled),n_folds=5, random_state=1, shuffle=True)

In [17]:
len(X_scaled)

97230

In [18]:
lr_gs = {'C': np.power(10.0, np.arange(-4, 2, 1))}
clf_lr = LogisticRegression(random_state = 1, verbose = 0)
clf_lr_gs = GridSearchCV(clf_lr, 
                               lr_gs,
                               cv = kf,
                               scoring = 'roc_auc'
                               )
clf_lr_gs.fit(X_scaled, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=1),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-04,   1.00000e-03,   1.00000e-02,   1.00000e-01,
         1.00000e+00,   1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

Довольно долгая процедура, но из-за того, что проверялось много разных C

В целом, логистическая регрессия работает быстрее, чем градиентный бустинг

In [19]:
y_score_lr = clf_lr_gs.decision_function(X_scaled)

print(clf_lr_gs.best_params_)
print(roc_auc_score(y, y_score_lr))


{'C': 0.01}
0.717857407964


Получаем значение метрики больше, чем при градиентном бустинге практически на 0.02

#### Проверим, насколько изменится качество, если убрать категориальные признаки

In [20]:
_X_cat = X.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)
_X_cat_scaled = scaler.fit_transform(_X_cat)

И выполним процедуру еще раз:

In [23]:
clf_logreg_grid = clf_lr_gs

In [24]:
clf_logreg_grid.fit(_X_cat_scaled, y)

y_cat = clf_logreg_grid.decision_function(_X_cat_scaled)

print(clf_logreg_grid.best_params_)
print(roc_auc_score(y, y_cat))
y_cat = clf_logreg_grid.decision_function(_X_cat_scaled)

print(clf_logreg_grid.best_params_)
print(roc_auc_score(y, y_cat))

{'C': 0.01}
0.717716874723
{'C': 0.01}
0.717716874723


Получаем незначительные отличия, ибо бессмысленно сравнивать грубо говоря айди героев, которые не имеют смысла с точки зрения математики.

#### Чуть не забыл, количество героев

Посмотрим, какие герои были выбраны в играх, а также, какой id максимальный -- чтобы знать, сколько героев

In [25]:
hero_id = []
picks = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
         'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for hero in picks:
    hero_id.extend(np.unique(X[hero]))
    
len(np.unique(hero_id))
print(np.unique(hero_id))

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 109 110 112]


Получаем, что всего героев **112**, однако в нашей выборке **108**. Это связано с тем, что в Dota 2, иногда выходят новые герои. Но также очень часто бывают ситуации, когда эти герои несбалансированы относительно остальных персонажей. Именно поэтому новых героев добавляют в *Captains mode* не сразу, а спустя какое-то время. Вероятно, 4 героя относятся к такой категории

#### Проверим "мешок слов"

In [26]:
N = 112
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_in_bag = np.hstack((_X_cat_scaled,X_pick))

In [27]:
clf_logreg_grid.fit(X_in_bag, y)

y_bag = clf_logreg_grid.decision_function(X_in_bag)

print(clf_logreg_grid.best_params_)
print(roc_auc_score(y, y_bag))

{'C': 0.10000000000000001}
0.754292733042


Заметно, что использование "мешка слов" ведет к довольно значительному увеличению качества. Это логично, т.к. вероятность выигрыша командой игры зависит от выбранных героев -- некоторые герои имеют высокий "винрейт", что означет, если данный герой участвует в игре, то вероятность выигрыша высока.

Но человеческий фактор тоже очень важен, поэтому качество изменилось не совсем уж и значительно, если смотреть глобально. Для увеличения качества лучше будет добавить новые признаки, зависящие от ситуации в игре до 5 минуты

## Проверка тестовых данных

In [28]:
test_data = pd.read_csv('features_test.csv', index_col='match_id')

Приводим данные в порядок

In [29]:
X_test = test_data.iloc[:,1:]

In [30]:
X_test.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,0,93,4,1103,1089,8,0,1,9,102,...,0,12,247,-86,272,3,4,2,0,118
7,1,20,2,556,570,1,0,0,9,6,...,2,-29,168,-54,NaN,3,2,2,1,16
10,1,112,2,751,808,1,0,0,13,26,...,1,-22,46,-87,186,1,3,3,0,-34
13,1,27,3,708,903,1,1,1,11,91,...,2,-49,30,-89,210,3,4,2,1,-26
16,1,39,4,1259,661,4,0,0,9,93,...,0,36,180,-86,180,1,3,2,1,-33


In [31]:
X_test = X_test.fillna(0)

Так как "мешок слов" хорошо себя показал, то будем использовать его. (Для первой попытки на kaggle использую только данные по заданию, без придумывания фич) 

In [32]:
X_pick_test = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

Проведем нормализацию без категориальных признаков, а затем соберем всё в кучу (как делали выше)

In [33]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
             'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for i in categ_features:
    X_test.drop(i,inplace=True,axis=1)
X_scaled_test = scaler.fit_transform(X_test)

In [34]:
X_in_bag_test = np.hstack((X_scaled_test, X_pick_test))

Проделав действия, аналогично тем, которые мы делали для тренировочной выборки, получаем предсказание на тестовых данных:

In [35]:
_y_cat_test = clf_logreg_grid.best_estimator_.predict_proba(X_in_bag_test)


Сделаем из предсказаний табличку с колонками 0 и 1:

In [36]:
_y_cat_test = pd.DataFrame(_y_cat_test)
_y_cat_test.mean()

0    0.481801
1    0.518199
dtype: float64

Получаем, что у **Radiant** вероятность выигрыша больше. Предположу, что это из-за немного разных расположений спаунов у крипов, а также разных расположений "легких" и "тяжелых" линий по отношению к Рошану (но это уже тонкости игры, и вряд ли для профессионалов они на что-то влияют)

Минимальное и максимальное значения вероятностей победы **Radiant**:

Теперь все складываем в таблицу **_result_** для изготовления _.csv_ файла и отправки на Kaggle (с небольшим костылем):

In [37]:
result = _y_cat_test
result.columns = ['match_id', 'radiant_win']
result.index = X_test.index
result = result.drop(labels='match_id', axis=1)

In [38]:
result.head()

,radiant_win
match_id,
6,0.836036
7,0.777719
10,0.201795
13,0.870913
16,0.261544


Минимальная и максимальная вероятности победы **Radiant**:

In [39]:
print((min(result['radiant_win'])))
print (max(result['radiant_win']))

0.00850297088461
0.996617560951


In [239]:
result.to_csv('answers_to_kaggle.csv',
             index = True)

## Конец

## P.S.
Первая попытка на Kaggle: **0.75485**

Для доработки стоит посмотреть "сырые" данные, а также погенерировать фичи хотя бы из исходных, часто бывает, что какие-то комбинации могут увеличить точность на какой-то процент.